#P3 Data Wrangling with MongoDB

###Map Area: San Francisco, CA

#0. Procedure
###Count types of tags in the dataset

To get a rough sense of the data

In [12]:
print count_tags('san-francisco.osm')

{'node': 1410191, 'nd': 1677325, 'bounds': 1, 'member': 26246, 'tag': 949435, 'relation': 1687, 'way': 154315, 'osm': 1}


###Count number of tag types

- lower, lower_colon and problemchars

In [19]:
tag_types_dict = process_map('san-francisco.osm')

In [5]:
# most of them look like seamark (nautical chart standard), they can be ignored.

###Auditing Postal codes and Street names

###Postal codes

- whitespaces --> trim
- CA --> truncate the front
- extension --> truncate the back
- 4 digit, 3 digit --> drop

###Street types

###Some invalid numeric street names

- house number in the wrong place
- 'PM'

###A's

- Abenue
- variants of avenue

###B's

- variants of blvd and broadway

###C's

- variants of crescent and court

###D's

- variants of drive

###H's

- variants of highway

###L's

- variants of lanes and plaza

###R's

- variants of road 

###S's 

- variants of streets

###Street types cleaning

{ "Ave.": "Ave",
            "Abenue": "Ave",
            "ave": "Ave",
            "avenue": "Ave",
            "Blvd,": "Blvd",
            "Blvd.": "Blvd",
            "Boulavard": "Blvd",
            "Boulvard": "Blvd",
            "broadway": "Broadway",
            "bush": "Bush",
            "Cres": "Crescent",
            "Ctr": "Center",
            "Dr.": "Dr",
            "Hwy": "Highway",
            "Ln.": "Ln",
            "Plz": "Plaza",
            "parkway": "Parkway", 
            "Rd.": "Rd",
            "Steet": "St",
            "St.": "St",
            "st": "St",
            "street": "St",
            "square": "Square",
            "sutter": "Sutter" }

###Converting to JSON and preparing for MongoDB 

- step is done in data_prep.py

#1. Problems encountered in the map

After thoroughly auditing the area, there are two main problems with the data:

- different variation of street names 
- different variation of postal codes


###Different variation of streetnames
The different variation of streetnames are programmatically cleaned as indicated by the mapping.


###Different variation of postal codes
The different variation of postal codes are programmatically cleaned.

#2. Data overview

The following section contains some basic statistics about the dataset and the queries used to gather them:

Date of data set: as of Dec 31, 2013

Source of data set: http://metro.teczno.com/#san-francisco

Reason: Living in SF

File sizes:

- san-francisco.osm.json = 359.9MB
- san-francisco.osm = 326.2MB

###Sort postcodes by count, descending

In [6]:
#     pipeline = [{"$match": {"address.postcode": {"$exists": 1}}}, 
#                 {"$group": {"_id": "$address.postcode", "count": {"$sum": 1}}}, 
#                 {"$sort": {"count":-1}}]

# postal codes look mostly correct. some empty strings are the inappropriate ones 
# that are taken out

### Sort cities by count, descending

In [7]:
#     pipeline = [{"$match":{"address.city":{"$exists":1}}}, 
#                 {"$group":{"_id":"$address.city", "count":{"$sum":1}}}, 
#                 {"$sort":{"count":-1}}]
    
# Several of the cities have several mispellings/variations: 
# San Francisco, Berkeley, Oakland

# Some unexpected ones: 157, 155

###Number of documents

In [9]:
#     result = db.sf1.find().count()
    
# Number of docs: 1564506

###Number of nodes

In [10]:
#     result = db.sf1.find({"type":"node"}).count()

# Number of nodes: 1410114 

###Number of ways

In [11]:
#     result = db.sf1.find({"type":"way"}).count()

# Number of ways: 154289

###Number of unique users

In [12]:
#     result = len(db.sf1.distinct("created.user"))

# Number of unique users: 1288

###Top 1 contributing user

In [18]:
#     pipeline = [{"$group":{"_id": "$created.user", "count":{"$sum": 1}}},
#                 {"$sort":{"count": -1}}, {"$limit": 1}]

[{u'_id': u'oldtopos', u'count': 334076}]


###Number of users having 1 post

In [19]:
#     pipeline = [{"$group":{"_id":"$created.user", "count":{"$sum": 1}}}, 
#                 {"$group":{"_id":"$count", "num_users":{"$sum": 1}}}, 
#                 {"$sort":{"_id": 1}}, {"$limit": 1}]

[{u'_id': 1, u'num_users': 292}]


#3. Additional Ideas

- User contribution is pretty skewed but has a healthy base of contributing users (~200 users contribute to 99% of the content)

- Among the top 500 contributors, most of the contributions are quite old. This might be normal or not (domain knowledge of how quickly SF is developing is required)

- To assess if the contributions are up to date, we can cross-validate the objects from the map with the objects/spots obtained from the Google Map API. You can craft an XML message to send to the Google API and get the coordinates in return. You can then validate the coordinates returned with the coordinates you have [source] (http://stackoverflow.com/questions/682093/address-validation-using-google-maps-api). This is not a terribly involved effort and would lead to improvement in benchmarking the data quality.

In [20]:
#     pipeline = [{"$group":{"_id": "$created.user", "count":{"$sum": 1}}},
#                 {"$sort":{"count": -1}}, {"$limit": 10}]

[{u'_id': u'oldtopos', u'count': 334076},
 {u'_id': u'KindredCoda', u'count': 144613},
 {u'_id': u'osmmaker', u'count': 140043},
 {u'_id': u'DanHomerick', u'count': 119462},
 {u'_id': u'nmixter', u'count': 77785},
 {u'_id': u'woodpeck_fixbot', u'count': 46008},
 {u'_id': u'StellanL', u'count': 43335},
 {u'_id': u'oba510', u'count': 38785},
 {u'_id': u'dchiles', u'count': 38472},
 {u'_id': u'Speight', u'count': 30346}]


###User contribution demographics

In [22]:
#     pipeline = [{"$group":{"_id": "$created.user", "count":{"$sum": 1}}},
#                 {"$sort":{"count": -1}}]

In [50]:
for j in xrange(50, 550, 50):
    top = sum([i['count'] for i in user_contribution[0:j]])
    whole = float(db.sf1.find().count())
    print 'Top {} users contribute to {} percent of content.'.format(j, top/whole)

Top 50 users contribute to 0.903069722967 percent of content.
Top 100 users contribute to 0.958983858164 percent of content.
Top 150 users contribute to 0.977836454446 percent of content.
Top 200 users contribute to 0.986199477663 percent of content.
Top 250 users contribute to 0.990499237459 percent of content.
Top 300 users contribute to 0.993142244261 percent of content.
Top 350 users contribute to 0.994950482772 percent of content.
Top 400 users contribute to 0.996272944942 percent of content.
Top 450 users contribute to 0.997200394246 percent of content.
Top 500 users contribute to 0.997874089329 percent of content.


In [63]:
#     pipeline = [{"$group":{"_id": {"timestamp": "$created.timestamp", 
#                     "user": "$created.user"},
#                 "count":{"$sum": 1}}}]

In [208]:
user_time_sums.set_index('user').loc[x[0:500]].sum(0)
# among the top 500 contributors most of the contributions are quite old

year
2007      4101
2008     67897
2009    195716
2010    140365
2011    409696
2012    292355
2013    451050
dtype: float64

###Top 10 amenities

In [209]:
#     pipeline = [{"$match": {"amenity":{"$exists":1}}}, 
#                 {"$group": {"_id":"$amenity", "count":{"$sum": 1}}}, 
#                 {"$sort": {"count": -1}}, {"$limit": 10}]

# surprising to see parking is the top 1 result. usually tough to find parking in SF!

[{u'_id': u'parking', u'count': 2958},
 {u'_id': u'restaurant', u'count': 1857},
 {u'_id': u'school', u'count': 1338},
 {u'_id': u'place_of_worship', u'count': 1064},
 {u'_id': u'fire_hydrant', u'count': 698},
 {u'_id': u'post_box', u'count': 588},
 {u'_id': u'cafe', u'count': 574},
 {u'_id': u'bench', u'count': 445},
 {u'_id': u'fast_food', u'count': 436},
 {u'_id': u'drinking_water', u'count': 333}]


###Top 10 religions

In [211]:
#     pipeline = [{"$match": {"amenity": {"$exists": 1}, "amenity" : "place_of_worship"}},
#                 {"$group": {"_id": "$religion", "count": {"$sum": 1}}},
#                 {"$sort": {"count": -1}}, {"$limit": 10}]
    
# Surprise to see scientologist on the list

[{u'_id': u'christian', u'count': 979},
 {u'_id': None, u'count': 45},
 {u'_id': u'buddhist', u'count': 16},
 {u'_id': u'jewish', u'count': 14},
 {u'_id': u'muslim', u'count': 4},
 {u'_id': u'unitarian', u'count': 2},
 {u'_id': u'scientologist', u'count': 2},
 {u'_id': u'shinto', u'count': 1},
 {u'_id': u'unitarian_universalist', u'count': 1}]


###Top 10 cuisines

In [212]:
#     pipeline = [{"$match": {"amenity": {"$exists":1}, "amenity": "restaurant"}}, 
#                 {"$group": {"_id": "$cuisine", "count": {"$sum": 1}}},        
#                 {"$sort": {"count": -1}}, {"$limit": 10}]
    
# Semi-surprise to see the mexican genre take the top spot

[{u'_id': None, u'count': 744},
 {u'_id': u'mexican', u'count': 135},
 {u'_id': u'pizza', u'count': 94},
 {u'_id': u'italian', u'count': 90},
 {u'_id': u'chinese', u'count': 85},
 {u'_id': u'japanese', u'count': 70},
 {u'_id': u'thai', u'count': 70},
 {u'_id': u'american', u'count': 59},
 {u'_id': u'burger', u'count': 54},
 {u'_id': u'indian', u'count': 47}]


##Conclusion

After this review of the data, it's clear that the San Francisco area is incomplete, although things that can be programmatically cleaned are cleaned. One salient feature is that most of the contributions from the contributors are quite old. Given the pace of development in SF, I'd suspect there would be more recent developments that are not captured.